In [4]:
# 골라놓은 알약
SRC_DIR = 'E:/Download_informatica30/경구약제 이미지 데이터/01.데이터/1.Training/원천데이터'  # '/경구약제조합 5000종/TL_1_조합/K-000250-000573-002483-006192_json/K-000250/K-000250-000573-002483-006192_0_2_0_2_75_000_200.json'
DST_DIR = './DATASET'
DST_COMB_DIR = './DATASET/COMBI' 

SEL_DRUG = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
NO = [i for i in range(31)]

CLS_N2D = dict(zip(NO, SEL_DRUG))
CLS_D2N = dict(zip(SEL_DRUG, NO))

In [5]:
import pandas as pd

df_sel_drug = pd.read_csv('./DRUG_INFO/선택한알약이미지정보.csv')

df_sel_drug.head(2)

,drug_N,drug_code,dl_name,dl_name_en,dl_material,dl_material_en,bbox,dl_company,dl_company_en,di_class_no,di_etc_otc_code,imgfile,json_path,Combi
0,K-001562,644700590,미노씬캡슐 50mg,Minocin Cap. 50mg,미노사이클린염산염,Minocycline Hydrochloride,326|433|307|321,에스케이케미칼(주),Sk Chemicals,"[06150]주로 그람양성, 음성균, 리케치아, 비루스에 작용하는 것",전문의약품,K-001562_0_2_0_0_70_000_200.png,/단일경구약제 5000종/TL_23_단일/K-001562_json/K-001...,TL_23_단일
1,K-001562,644700590,미노씬캡슐 50mg,Minocin Cap. 50mg,미노사이클린염산염,Minocycline Hydrochloride,293|464|367|264,에스케이케미칼(주),Sk Chemicals,"[06150]주로 그람양성, 음성균, 리케치아, 비루스에 작용하는 것",전문의약품,K-001562_0_2_0_0_70_020_200.png,/단일경구약제 5000종/TL_23_단일/K-001562_json/K-001...,TL_23_단일


In [ ]:
for file in files :
    if file in ['.DS_Store', 'COMBI']  : continue
    
    info = df_sel_drug.loc[df_sel_drug['imgfile']==file, ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
    drug_N = info.loc[0, 'drug_N']
    bbox = info.loc[0, 'bbox']
    coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
    

In [7]:
# 모든 파일을 all_files 리스트로...
import os

drug_dir = os.listdir('./DATASET/')

cls_name = []
for drug_N in SEL_DRUG : # .DS_Store' 제외
    info = df_sel_drug.loc[df_sel_drug['drug_N']==drug_N, ['dl_name_en', 'drug_N']].reset_index(drop=True) # 약 정보 찾기
    drug_N = info.loc[0, 'drug_N']
    dl_name_en = info.loc[0, 'dl_name_en']
    cls_name.append(dl_name_en)
    
cls_name

f = open('cls_name.txt', 'w').write(str(cls_name))
f.close()

AttributeError: 'int' object has no attribute 'close'

### 단일 경구약 이미지 정답파일 생성

In [5]:
# 단일 이미지 정답 만들기
import os

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

drug_dir = os.listdir('./DATASET/')

for drug_N in drug_dir[1:] : # .DS_Store' 제외
    files = os.listdir('./DATASET/'+drug_N)
    for file in files :
        if file in ['.DS_Store', 'COMBI']  : continue
        
        info = df_sel_drug.loc[df_sel_drug['imgfile']==file, ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        drug_N = info.loc[0, 'drug_N']
        bbox = info.loc[0, 'bbox']
        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        
        yolo_bbox = convert_bbox_coco2yolo(976, 1280, coco_bbox)

        file = open('./DATASET/'+drug_N+'/'+file.replace('png', 'txt'), 'w')
        
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")
        file.close()

NotADirectoryError: [WinError 267] 디렉터리 이름이 올바르지 않습니다: './DATASET/COMBIK-005391_K-005391_K-002052_K-002052_K-005391_K-005391_K-002052_K-002052.png'

756

### 이미지 배치 #1 - 4장 배치

In [49]:
# 이미지 배치하기
import random
from PIL import Image

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

def convert_bbox_pill_crop(bbox, margin):
    x_tl, y_tl, w, h = bbox

    x1, y1 = x_tl - margin, y_tl - margin
    x2, y2 = x_tl + w + margin, y_tl + h + margin

    return (x1, y1, x2, y2), (w + margin * 2, h + margin * 2)


def pill_allocation4(drug_L, margin) :
    pos = [(0, 0, 488, 640), (488, 0, 960, 640), (0, 640, 488, 1280), (488, 640, 976, 1280)]

    drug_N_L = [drug[0:8] for drug in drug_img_L]        
    filename = '_'.join(drug_N_L)
    
    new_img = Image.new("RGB", (976, 1280), "white") # 빈 캔버스 만들기
    file = open(DST_COMB_DIR + '/' + filename + '.txt', 'w') # 정답 파일 열기
    for i in range(len(drug_L)) : 
        info = df_sel_drug.loc[df_sel_drug['imgfile']==drug_L[i], ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        
        drug_N = info.loc[0, 'drug_N'] # 약 코드(K-?????)
        bbox = info.loc[0, 'bbox']

        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        crop_bbox, crop_wh = convert_bbox_pill_crop(coco_bbox, margin)       # tuple (x1, y1, x2, y2), (width, height)

        if crop_wh[0] >= pos[0][2] or crop_wh[1] >= pos[0][3] : continue # crop_wh가 셀 크기보다 큰 경우

        pill_img = Image.open(DST_DIR + '/' + drug_N + '/' + drug_L[i]) # 약 이미지 불러오기
        pill_img = pill_img.crop(crop_bbox)   # crop_bbox 영역만큼 crop

        adjust_pos = (random.randint(pos[i][0], pos[i][2]-crop_wh[0]), random.randint(pos[i][1], pos[i][3]-crop_wh[1]))
        adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        yolo_bbox = convert_bbox_coco2yolo(976, 1280, adjust_coco_bbox)

        # print(drug_cls, coco_bbox, adjust_coco_bbox, yolo_bbox)
        new_img.paste(im=pill_img, box=adjust_pos)
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")
        print(bbox)

    new_img.save(DST_COMB_DIR + '/' + filename + '.png')
    file.close()


# 초기설정
image_count = 4
image_magin = 20

drug_img_L = random.sample(all_files, image_count)
# drug_img_L = ['K-001562_0_2_0_0_70_000_200.png', 'K-001562_0_2_0_0_70_020_200.png', 'K-001562_0_2_0_0_70_040_200.png', 'K-001562_0_2_0_0_70_060_200.png'] # 시험용
pill_allocation4(drug_img_L, image_magin)


326|433|307|321
293|464|367|264
276|503|399|189
267|500|396|179


### 이미지 배치 #2 - 6장 배치

In [75]:
# 이미지 배치하기
import random
from PIL import Image

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

def convert_bbox_pill_crop(bbox, margin):
    x_tl, y_tl, w, h = bbox

    x1, y1 = x_tl - margin, y_tl - margin
    x2, y2 = x_tl + w + margin, y_tl + h + margin

    return (x1, y1, x2, y2), (w + margin * 2, h + margin * 2)


def pill_allocation6(drug_L, margin) :
    pos = [(0, 0, 488, 426), (488, 0, 976, 426), (0, 426, 488, 852), (488, 426, 976, 852), (0, 852, 488, 1280), (488, 852, 976, 1280)]

    drug_N_L = [drug[0:8] for drug in drug_img_L]        
    filename = '_'.join(drug_N_L)
    
    new_img = Image.new("RGB", (976, 1280), "white") # 빈 캔버스 만들기
    file = open(DST_COMB_DIR + '/' + filename + '.txt', 'w') # 정답 파일 열기
    for i in range(len(drug_L)) : 
        info = df_sel_drug.loc[df_sel_drug['imgfile']==drug_L[i], ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        
        drug_N = info.loc[0, 'drug_N'] 
        bbox = info.loc[0, 'bbox']

        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        crop_bbox, crop_wh = convert_bbox_pill_crop(coco_bbox, margin)       # tuple (x1, y1, x2, y2), (width, height)

        pill_img = Image.open(DST_DIR + '/' + drug_N + '/' + drug_L[i]) # 약 이미지 불러오기
        pill_img = pill_img.crop(crop_bbox)   # crop_bbox 영역만큼 crop

        if crop_wh[0] >= pos[0][2] or crop_wh[1] >= pos[0][3] : continue # crop_wh가 셀 크기보다 큰 경우

        adjust_pos = (random.randint(pos[i][0], pos[i][2]-crop_wh[0]), random.randint(pos[i][1], pos[i][3]-crop_wh[1]))
        adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        yolo_bbox = convert_bbox_coco2yolo(976, 1280, adjust_coco_bbox)

        # print(drug_cls, coco_bbox, adjust_coco_bbox, yolo_bbox)
        new_img.paste(im=pill_img, box=adjust_pos)
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")

    new_img.save(DST_COMB_DIR + '/' + filename + '.png')
    file.close()


# 초기설정
image_count = 6
image_magin = 20

drug_img_L = random.sample(all_files, image_count)
pill_allocation6(drug_img_L, image_magin)

### 이미지 배치 #3 - 12장 배치(이미지가 삽입되지 않는 경우 무시)

In [85]:
# 이미지 배치하기
import os
import random
from PIL import Image

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]


def convert_bbox_pill_crop(bbox, margin):
    x_tl, y_tl, w, h = bbox

    x1, y1 = x_tl - margin, y_tl - margin
    x2, y2 = x_tl + w + margin, y_tl + h + margin

    return (x1, y1, x2, y2), (w + margin * 2, h + margin * 2)


def pill_allocation12_under(drug_L, margin) :
    pos = [(0, 0, 325, 320), (325, 0, 650, 320), (650, 0, 976, 320), (0, 320, 325, 640), (325, 320, 650, 640), (650, 320, 976, 640), (0, 640, 325, 960), (325, 640, 650, 960), (650, 640, 976, 960), (0, 960, 325, 1280), (325, 960, 650, 1280), (650, 960, 976, 1280)]

    drug_N_L = [drug[0:8] for drug in drug_img_L]        
    #filename = '_'.join(drug_N_L)
    
    new_img = Image.new("RGB", (976, 1280), "white") # 빈 캔버스 만들기
    file = open(DST_COMB_DIR + '/temp.txt', 'w') # 정답 파일 열기
    #file = open(DST_COMB_DIR + '/' + filename + '.txt', 'w') # 정답 파일 열기

    available_drug_L = []
    for i in range(len(drug_L)) : 
        info = df_sel_drug.loc[df_sel_drug['imgfile']==drug_L[i], ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        
        drug_N = info.loc[0, 'drug_N'] # 약 코드(K-?????)
        bbox = info.loc[0, 'bbox']

        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        crop_bbox, crop_wh = convert_bbox_pill_crop(coco_bbox, margin)       # tuple (x1, y1, x2, y2), (width, height)

        pill_img = Image.open(DST_DIR + '/' + drug_N + '/' + drug_L[i]) # 약 이미지 불러오기
        pill_img = pill_img.crop(crop_bbox)   # crop_bbox 영역만큼 crop

        if crop_wh[0] >= pos[0][2] or crop_wh[1] >= pos[0][3] : continue # crop_wh가 셀 크기보다 큰 경우

        available_drug_L.append(drug_N)

        adjust_pos = (random.randint(pos[i][0], pos[i][2]-crop_wh[0]), random.randint(pos[i][1], pos[i][3]-crop_wh[1]))
        adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        yolo_bbox = convert_bbox_coco2yolo(976, 1280, adjust_coco_bbox)

        # print(drug_cls, coco_bbox, adjust_coco_bbox, yolo_bbox)
        new_img.paste(im=pill_img, box=adjust_pos)
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")

    file.close()
    
    filename = '_'.join(available_drug_L)
    new_img.save(DST_COMB_DIR + filename + '.png')    
    os.rename('temp.txt', filename + '.txt')


# 초기설정
image_count = 12
image_magin = 20

drug_img_L = random.sample(all_files, image_count)
pill_allocation12_under(drug_img_L, image_magin)

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'temp.txt' -> 'K-005391_K-005391_K-002052_K-002052_K-005391_K-005391_K-002052_K-002052.txt'

### 이미지배치 #5 - 4장 배치(이미지 크기 임의 확대)

In [69]:
# 이미지 배치하기
import random
from PIL import Image

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]

def convert_bbox_pill_crop(bbox, margin):
    x_tl, y_tl, w, h = bbox

    x1, y1 = x_tl - margin, y_tl - margin
    x2, y2 = x_tl + w + margin, y_tl + h + margin

    return (x1, y1, x2, y2), (w + margin * 2, h + margin * 2)


def pill_allocation4_size_up(drug_L, margin) :
    pos = [(0, 0, 488, 640), (488, 0, 960, 640), (0, 640, 488, 1280), (488, 640, 976, 1280)]

    drug_N_L = [drug[0:8] for drug in drug_img_L]        
    filename = '_'.join(drug_N_L)
    
    new_img = Image.new("RGB", (976, 1280), "white") # 빈 캔버스 만들기
    file = open(DST_COMB_DIR + '/' + filename + '.txt', 'w') # 정답 파일 열기
    for i in range(len(drug_L)) : 
        info = df_sel_drug.loc[df_sel_drug['imgfile']==drug_L[i], ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        
        drug_N = info.loc[0, 'drug_N'] # 약 코드(K-?????)
        bbox = info.loc[0, 'bbox']

        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        crop_bbox, crop_wh = convert_bbox_pill_crop(coco_bbox, margin)       # tuple (x1, y1, x2, y2), (width, height)

        print(coco_bbox, end=' ')

        pill_img = Image.open(DST_DIR + '/' + drug_N + '/' + drug_L[i]) # 약 이미지 불러오기
        pill_img = pill_img.crop(crop_bbox)   # crop_bbox 영역만큼 crop

        # enlarge = 3 # 무조건 확대
        enlarge = random.randint(0, 10) # 홀수면 확대, 짝수면 그대로

        if enlarge % 2 == 1 : 
            w_rate = pos[0][2] / crop_wh[0]   # 1 초과이면, 약보다 셀이 큰 경우
            h_rate = pos[0][3] / crop_wh[1]

            size_up_rate = min(w_rate, h_rate) - 0.08 # 확대할 비율 (0.05은 여유분)

            # 크기 조절
            crop_wh = (int(crop_wh[0] * size_up_rate), int(crop_wh[1] * size_up_rate))
            pill_img = pill_img.resize(crop_wh) # 비율에 맞게 이미지 resize

            # 정답 위치 조절
            coco_bbox = (coco_bbox[0], coco_bbox[1], int(coco_bbox[2] * size_up_rate), int(coco_bbox[3] * size_up_rate))

            print(size_up_rate, coco_bbox, end=' ')

        adjust_pos = (random.randint(pos[i][0], pos[i][2]-crop_wh[0]), random.randint(pos[i][1], pos[i][3]-crop_wh[1]))  # 해당 셀의 x, y 위치 랜덤으로 지정

        if enlarge % 2 == 1 : 
            adjust_coco_bbox = (adjust_pos[0] + int(margin*size_up_rate),  adjust_pos[1] + int(margin*size_up_rate), int(coco_bbox[2])+1, int(coco_bbox[3])+1)
            # adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])
        else :
            adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        # adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        print(adjust_coco_bbox)

        yolo_bbox = convert_bbox_coco2yolo(976, 1280, adjust_coco_bbox)

        new_img.paste(im=pill_img, box=adjust_pos)
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")
        # print(bbox)

    new_img.save(DST_COMB_DIR + '/' + filename + '.png')
    file.close()
    new_img.show()


# 초기설정
image_count = 4
image_magin = 20

drug_img_L = random.sample(all_files, image_count)
# drug_img_L = ['K-001562_0_2_0_0_70_000_200.png', 'K-001562_0_2_0_0_70_020_200.png', 'K-001562_0_2_0_0_70_040_200.png', 'K-001562_0_2_0_0_70_060_200.png'] # 시험용
pill_allocation4_size_up(drug_img_L, image_magin)


[373, 448, 226, 378] (81, 74, 226, 378)
[425, 571, 191, 187] 2.0325541125541124 (425, 571, 388, 380) (530, 149, 389, 381)
[402, 497, 184, 191] (41, 911, 184, 191)
[387, 538, 196, 189] (685, 930, 196, 189)


### 이미지 배치 #6 - 12장 배치(이미지의 크기 임의 변경)

In [24]:
# 이미지 배치하기
import random
from PIL import Image

def convert_bbox_coco2yolo(img_width, img_height, bbox):
    x_tl, y_tl, w, h = bbox

    dw = 1.0 / img_width
    dh = 1.0 / img_height

    x_center = x_tl + w / 2.0
    y_center = y_tl + h / 2.0

    x = x_center * dw
    y = y_center * dh
    w = w * dw
    h = h * dh

    return [x, y, w, h]


def convert_bbox_pill_crop(bbox, margin):
    x_tl, y_tl, w, h = bbox

    x1, y1 = x_tl - margin, y_tl - margin
    x2, y2 = x_tl + w + margin, y_tl + h + margin

    return (x1, y1, x2, y2), (w + margin * 2, h + margin * 2)


def pill_allocation12_full(drug_L, margin) :
    pos = [(0, 0, 325, 320), (325, 0, 650, 320), (650, 0, 976, 320), (0, 320, 325, 640), (325, 320, 650, 640), (650, 320, 976, 640), (0, 640, 325, 960), (325, 640, 650, 960), (650, 640, 976, 960), (0, 960, 325, 1280), (325, 960, 650, 1280), (650, 960, 976, 1280)]

    drug_N_L = [drug[0:8] for drug in drug_img_L]        
    filename = '_'.join(drug_N_L)
    
    new_img = Image.new("RGB", (976, 1280), "white") # 빈 캔버스 만들기
    file = open(DST_COMB_DIR + '/' + filename + '.txt', 'w') # 정답 파일 열기
    for i in range(len(drug_L)) : 
        info = df_sel_drug.loc[df_sel_drug['imgfile']==drug_L[i], ['drug_N', 'bbox']].reset_index(drop=True) # 약 정보 찾기
        
        drug_N = info.loc[0, 'drug_N']
        bbox = info.loc[0, 'bbox']

        coco_bbox = list(map(int, bbox.split('|')))                         # list [x1, y1, width, height]
        crop_bbox, crop_wh = convert_bbox_pill_crop(coco_bbox, margin)       # tuple (x1, y1, x2, y2), (width, height)

        pill_img = Image.open(DST_DIR + '/' + drug_N + '/' + drug_L[i]) # 약 이미지 불러오기
        pill_img = pill_img.crop(crop_bbox)   # crop_bbox 영역만큼 crop

        w_rate = pos[0][2] / crop_wh[0]   # 0 미만이면, 셀보다 큰 경우
        h_rate = pos[0][3] / crop_wh[1]

        if w_rate < 1 or h_rate < 1 : 
            size_down_rate = min(w_rate, h_rate) # 초과되는 비율 계산

            # 크기 조절
            crop_wh = (int(crop_wh[0] * size_down_rate), int(crop_wh[1] * size_down_rate))
            pill_img = pill_img.resize(crop_wh) # 비율에 맞게 resize

            # 정답 위치 조절
            coco_bbox = (coco_bbox[0], coco_bbox[1], int(coco_bbox[2] * size_down_rate), int(coco_bbox[3] * size_down_rate))
            

        adjust_pos = (random.randint(pos[i][0], pos[i][2]-crop_wh[0]), random.randint(pos[i][1], pos[i][3]-crop_wh[1]))

        if w_rate < 1 or h_rate < 1 : 
            adjust_coco_bbox = (adjust_pos[0] + int(margin*size_down_rate),  adjust_pos[1] + int(margin*size_down_rate), int(coco_bbox[2])+1, int(coco_bbox[3])+1)
        else :
            adjust_coco_bbox = (adjust_pos[0]+margin,  adjust_pos[1]+margin, coco_bbox[2], coco_bbox[3])

        yolo_bbox = convert_bbox_coco2yolo(976, 1280, adjust_coco_bbox)

        new_img.paste(im=pill_img, box=adjust_pos)
        file.writelines(f"{CLS_D2N[drug_N]} {str(yolo_bbox).replace('[', '').replace(']', '').replace(',', '')}\n")

    new_img.save(DST_COMB_DIR + '/' + filename + '.png')
    file.close()
    # new_img.show()


# 초기설정
image_count = 12
image_magin = 20

drug_img_L = random.sample(all_files, image_count)
pill_allocation12_full(drug_img_L, image_magin)

In [ ]:
glob.glob(wildcard) 
- 유닉스 경로명 패턴 스타일로 파일 목록을 얻을 수 있다.
 
os.listdir(path) 
- 지정된 디렉토리의 전체 파일 목록을 얻을 수 있다.
 
dircache.listdir(path) 
- os.listdir(path)와 동일한 파일 목록을 전달한다.
  path가 변경되지 않았을 때, dircache.listdir()은 다시 디렉토리 구조를 읽지 않고 이미 읽은 정보를 활용
 
dircache.annotate(head, list) - 일반 파일명과 디렉토리명을 구분해주는 함수
 

디렉토리 다루기
os.chdir(path) - 작업하고 있는 디렉토리 변경
os.getcwd() - 현재 프로세스의 작업 디렉토리 얻기
os.remove( filename or path ) - 파일이나 디렉토리 지우기
os.mkdir( path ) - 디렉토리 만들기
os.makedirs( path ) - 디렉토리 만들기와 동일하지만 /tmp/include/gl/temp 처럼 긴 경로를 한번에 만들어 준다.
os.path.abspath(filename) - 파일의 상대 경로를 절대 경로로 바꾸는 함수
os.path.exists(filename) - 주어진 경로의 파일이 있는지 확인하는 함수
os.curdir() - 현재 디렉토리 얻기
os.pardir() - 부모 디렉토리 얻기
os.sep() - 디렉토리 분리 문자 얻기. windows는 \ linux는 / 를 반환한다.
 


경로명 분리하기
os.path.basename(filename) - 파일명만 추출
os.path.dirname(filename) - 디렉토리 경로 추출
os.path.split(filename) - 경로와 파일명을 분리
os.path.splitdrive(filename) - 드라이브명과 나머지 분리 (MS Windows의 경우)
os.path.splitext(filename) - 확장자와 나머지 분리

In [8]:
def number_generator():
    yield 0
    yield 1
    yield 2
 
for i in number_generator():
    print(i)

0
1
2


: 